In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/csc3160

Mounted at /content/drive
/content/drive/MyDrive/csc3160


In [2]:
%pip install transformers tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.3 MB/s eta 0:00:00


In [ ]:
%ls

abstract/                     llama/            requirements.txt
CSC3160-119010153-XiangyiLi/  milestone2.ipynb  test_all_chatgpt.txt
csc3160.ipynb                 preprocess.py     test.py
dataset/                      proposal.ipynb    text_test.txt
image.png                     README.md         train.py


# Baseline RoBERTa Model (Given)

In [ ]:
# default settings and default test dataset
!python preprocess.py
!python train.py

2023-05-12 14:08:55.765391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were

In [ ]:
# test the baseline model on the expanded datset. 
!python test_with_args.py \
  1_to_5372.txt \
  output_br_1_to_5372.txt \
  best_model.pt

# Fine tuning the model

In [ ]:
!python preprocess_with_args.py \
  --input_file dataset/all_plus_until_5372.jsonl \
  --merged_file dataset/merged_plus_until_5372.json \
  --train_file dataset/train_plus_until_5372.json \
  --val_file dataset/val_plus_until_5372.json \
  --test_file dataset/test_plus_until_5372.json
!python train.py \
  --train_file dataset/train_plus_until_5372.json \
  --val_file dataset/val_plus_until_5372.json \
  --test_file dataset/test_plus_until_5372.json \
  --model_path fine_tuned_with_roberta.pt

2023-05-12 15:17:22.023908: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were

# Fine-tuned with Dolly data

In [7]:
!python preprocess_with_args.py \
  --input_file dataset/all_plus_until_5372.jsonl \
  --merged_file dataset/merged_plus_until_5372.json \
  --train_file dataset/train_plus_until_5372.json \
  --val_file dataset/val_plus_until_5372.json \
  --test_file dataset/test_plus_until_5372.json

In [13]:
import json

# Open the files
with open('questions_base.txt', 'r') as q_file, open('dolly-1-675.txt', 'r') as d_file, open('dataset/train_plus_until_5372.json', 'r') as old_file:

    questions = q_file.readlines()
    answers = d_file.readlines()

    # # Check if number of lines match
    # if len(questions) != len(answers):
    #     raise ValueError("Number of lines in files do not match")

    # Load the old data
    old_data = json.load(old_file)
    
    # Prepare new data
    new_data = []
    for i in range(len(answers)):
        new_data.append({
            "question": questions[i].strip(),
            "text": answers[i].strip(),
            "fake": 1
        })

# Combine old data and new data
all_data = old_data + new_data
print(len(all_data))
print(len(old_data))


# Write to new file
with open('dataset/train-with-cohere.json', 'w') as new_file:
    json.dump(all_data, new_file)

with open('questions_base.txt', 'r') as q_file, open('dolly-675-775.txt', 'r') as d_file, open('dataset/val_plus_until_5372.json', 'r') as old_file:

    questions = q_file.readlines()
    answers = d_file.readlines()

    # # Check if number of lines match
    # if len(questions) != len(answers):
    #     raise ValueError("Number of lines in files do not match")

    # Load the old data
    old_data = json.load(old_file)
    
    # Prepare new data
    new_data = []
    for i in range(len(answers)):
        new_data.append({
            "question": questions[i].strip(),
            "text": answers[i].strip(),
            "fake": 1
        })

# Combine old data and new data
all_data = old_data + new_data

# Write to new file
with open('dataset/val-with-cohere.json', 'w') as new_file:
    json.dump(all_data, new_file)


81524
80851


In [3]:
!python train.py \
  --train_file dataset/train-with-cohere.json \
  --val_file dataset/val-with-cohere.json \
  --test_file dataset/test_plus_until_5372.json \
  --model_path fine_tuned_with_roberta_with_dolly.pt

avg_loss: 0.0087:  30% 3054/10191 [06:43<17:14,  6.90it/s]Traceback (most recent call last):
  File "/content/drive/MyDrive/csc3160/train.py", line 240, in <module>
    trainer.run_training()
  File "/content/drive/MyDrive/csc3160/train.py", line 196, in run_training
    train_loss = self.train_model(train_loader)
  File "/content/drive/MyDrive/csc3160/train.py", line 132, in train_model
    loss.backward()
  File "/usr/local/lib/python3.10/dist-packages/torch/_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py", line 200, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt
avg_loss: 0.0087:  30% 3054/10191 [06:43<15:44,  7.56it/s]
^C


In [ ]:
# fr roberta
!python test_with_args.py \
  test_all_chatgpt.txt \
  output_fr_baseline_chatgpt.txt \
  fine_tuned_with_roberta.pt

!python test_with_args.py \
  1_to_5372.txt \
  output_fr_1_to_5372.txt \
  fine_tuned_with_roberta.pt

!python test_with_args.py \
  5372_to_6185.txt \
  output_fr_5372_to_6185.txt \
  fine_tuned_with_roberta.pt


In [ ]:
# dolly, further generalizability test
!python test_with_args.py \
  dolly_from_1_to_875.txt \
  output_fr_dolly.txt \
  fine_tuned_with_roberta.pt

In [ ]:
# dolly with baseline
!python test_with_args.py \
  dolly_from_1_to_875.txt \
  output_br_dolly.txt \
  best_model.pt

In [5]:
# dolly with fr with dolly
!python test_with_args.py \
  dolly-775-875.txt \
  output_frd_dolly-775-875.txt \
  fine_tuned_with_roberta_with_dolly.pt

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
# fr 5372 to 6185 => 0.9124
!python test_with_args.py \
  5372_to_6185.txt \
  output_fr_5372_to_6185.txt \
  fine_tuned_with_roberta.pt

In [ ]:
# basline with cohere 30 data
!python test_with_args.py \
  cohere_from_1.txt \
  output_br_cohere.txt \
  best_model.pt

In [ ]:
# fr with cohere 30 data
!python test_with_args.py \
  cohere_from_1.txt \
  output_fr_cohere_from_1.txt \
  fine_tuned_with_roberta.pt

In [6]:
# fr (with dolly) with cohere 30 data
!python test_with_args.py \
  cohere_from_1.txt \
  output_frd_cohere_from_1.txt \
  fine_tuned_with_roberta_with_dolly.pt

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

# Bench with baseline

In [3]:
import json

# Read the JSON file
with open('dataset/test.json') as file:
    data = json.load(file)

# Extract the questions
texts = [entry['text'] for entry in data]


# Write the questions to a file
with open('answers_base.txt', 'w') as file:
    for text in texts:
        file.write(text.replace('\r', ' ').replace('\n', ' ') + '\n')


In [4]:
!wc -l answers_base.txt

17090 answers_base.txt


In [5]:
!python test_with_args.py \
  answers_base.txt \
  output_br_answers_base.txt \
  best_model.pt

!python test_with_args.py \
  answers_base.txt \
  output_fr_answers_base.txt \
  fine_tuned_with_roberta.pt

!python test_with_args.py \
  answers_base.txt \
  output_frd_answers_base.txt \
  fine_tuned_with_roberta_with_dolly.pt

Streaming output truncated to the last 5000 lines.
human
human
human
human
chatGPT
human
chatGPT
chatGPT
human
human
chatGPT
human
chatGPT
human
human
human
human
human
chatGPT
human
human
chatGPT
chatGPT
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
human
chatGPT
chatGPT
chatGPT
human
human
chatGPT
human
human
human
human
human
human
chatGPT
human
human
chatGPT
chatGPT
human
human
human
human
human
human
chatGPT
chatGPT
human
chatGPT
chatGPT
chatGPT
human
human
human
chatGPT
chatGPT
human
human
human
chatGPT
human
human
human
human
human
human
human
human
chatGPT
human
chatGPT
human
human
human
human
chatGPT
chatGPT
human
human
human
human
chatGPT
human
human
chatGPT
human
human
human
chatGPT
human
chatGPT
human
chatGPT
chatGPT
chatGPT
chatGPT
chatGPT
human
human
human
human
human
human
chatGPT
human
chatGPT
chatGPT
human
human
human
chatGPT
human
human
human
chatGPT
human
human
chatGPT
c

In [6]:
import json

# Open the JSON file
with open('dataset/test.json', 'r') as json_file:
    data = json.load(json_file)

# Extract the "fake" field and replace 1 and 0 with "chatGPT" and "human" respectively
truth_values = ['chatGPT' if item['fake'] == 1 else 'human' for item in data]

# Write the values to the new text file
with open('bl-test-truth.txt', 'w') as text_file:
    for value in truth_values:
        text_file.write(value + '\n')


In [7]:
# benchmark dataset
from sklearn.metrics import accuracy_score

# Load the true labels
with open('bl-test-truth.txt', 'r') as f:
    y_true = [line.strip() for line in f]

# List of your prediction files
prediction_files = ['output_br_answers_base.txt', 'output_fr_answers_base.txt', 'output_frd_answers_base.txt']

for file in prediction_files:
    # Load the predicted labels
    with open(file, 'r') as f:
        y_pred = [line.strip() for line in f]

    # Calculate and print accuracy score
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy for {file}: {accuracy}')


Accuracy for output_br_answers_base.txt: 0.9918080748976009
Accuracy for output_fr_answers_base.txt: 0.9366881217086015
Accuracy for output_frd_answers_base.txt: 0.9970743124634289
